# Imports

In [1]:
import mediapipe as mp

In [2]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import argparse

# ARGPARSE

In [3]:
args = argparse.ArgumentParser()

args.add_argument("--mode", default='webcam')
args.add_argument("--filePath", default=None)

arg, _ = args.parse_known_args()

In [4]:
print(arg.mode)
print(arg.filePath)

webcam
None


## Making output Directory if needed

In [10]:
output_directory_path = './Project3_output'
os.makedirs(output_directory_path,exist_ok=True)

## Face Detection using mediapipe

In [6]:
mp_face_detection = mp.solutions.face_detection

In [7]:
# model_selection = 0 means face is less than 2meters to cam and model_selection=1, means face is far away from camera
# confidence means higher the confidence higher the accuracy of assumption.

In [8]:
def process_image(image,face_detection):
    H,W,_ = image.shape
    rgb_img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    output = face_detection.process(rgb_img)
    if output.detections !=None:

        for detection in output.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box
            # coordinates
            xmin = bbox.xmin
            ymin = bbox.ymin
            width = bbox.width
            height = bbox.height

            # Absolute values of coordinates
            x_bottom_left = int(xmin*W)
            y_bottom_left = int(ymin*H)
            w = int(width*W)
            h = int(height*H)

            #drawing a rectangle over image
            rect = image.copy()
            rect = cv2.rectangle(rect,(x_bottom_left,y_bottom_left),(x_bottom_left+w,y_bottom_left+h),(0,255,0),5)

            blur_img = image.copy()
            blur_img[y_bottom_left:y_bottom_left+h,x_bottom_left:x_bottom_left+w] = cv2.blur(blur_img[y_bottom_left:y_bottom_left+h,x_bottom_left:x_bottom_left+w],(50,50))
        return blur_img

## Using argparse and process_image to blur our webcam image

In [9]:
with mp_face_detection.FaceDetection(model_selection=0,min_detection_confidence = 0.5) as face_detection: 
    if arg.mode == 'image':
        image_argpass = cv2.imread(arg.filePath)
        blur_img = process_image(image_argpass,face_detection)

        cv2.imwrite(os.path.join(output_directory_path,'output.png'),blur_img)

    elif arg.mode == 'webcam':
        cap = cv2.VideoCapture(0)
        ret,frame = cap.read()

        while ret:
            # if ret:
            blur_img = process_image(frame,face_detection)
            
            if blur_img is not None:
                frame_to_show = blur_img
            else:
                frame_to_show = frame
                
            cv2.imshow('No Face',frame_to_show)
            
            key = cv2.waitKey(10)

            if key & 0xFF == ord('t'):  # If 'q' is pressed, exit the loop
                break
                
            ret,frame = cap.read()

        cv2.destroyAllWindows()
        cap.release()
    pass